In [4]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets

pd.options.display.float_format = "{:,.2f}".format

In [23]:
INPUT_FILE = r"D:\Analysis\Plastic & Rubber - Part Family Wise.xlsx"

df = pd.read_excel(INPUT_FILE)
df.columns = df.columns.str.strip()

PART_FAMILY_COL = "Part Family"
PART_COL = "PartNo"
PRICE_COL = "PO Price"
SUPPLIER_COL = "Vendor"

df[PRICE_COL] = pd.to_numeric(df[PRICE_COL], errors="coerce")

display(df.head())

,Part Family,PartNo,PO Price,Technology,Plant,Vendor,Vehicle Model,Part Name,Part Type,Code,...,Tool Cost,Amortization Quantity (Tool Life),Tool Maintenance Cost (per pcs),Net Tool Cost,Other Cost Percentage,Any Other Cost,Effective Date,Basic Price,Net Cost Other Currency,Net Cost (INR)
0,Cover Handle Front,KE220419,71.67,Plastic,HOS,21677,IQUBE,COVER HANDLE FRONT,Component,RM-10001713,...,0.20,0.00,0.20,0.20,NaN,0.00,18/07/2023,79.02,0,79.02
1,Cover Handle Front,KE220419,71.67,Plastic,HOS,21677,IQUBE,COVER HANDLE FRONT,Component,RM-1000543,...,0.20,0.00,0.20,0.20,NaN,0.00,02/01/2023,78.97,0,78.97
2,Cover Handle Front,KL220029,49.50,Plastic,HP,21591,NTORQ,COVER HANDLE FRONT Unpainted,Component,RM-10001386,...,0.21,0.00,0.21,0.21,NaN,0.00,18/05/2021,50.75,0,50.75
3,Cover Handle Front,KL220029,49.50,Plastic,HP,21591,NTORQ,COVER HANDLE FRONT Unpainted,Component,RM-10001691,...,0.21,0.00,0.21,0.21,NaN,0.00,18/05/2021,50.82,0,50.82
4,Cover Handle Front,KL220029,49.50,Plastic,HP,21591,NTORQ,COVER HANDLE FRONT Unpainted,Component,RM-10002272,...,0.21,0.00,0.21,0.21,NaN,0.00,18/05/2021,50.44,0,50.44


In [ ]:
part_family_dd = widgets.Dropdown(
    options=sorted(df[PART_FAMILY_COL].dropna().unique()),
    description="Part Family:",
    layout=widgets.Layout(width="50%")
)

part_dd = widgets.Dropdown(
    options=[],
    description="Part No:",
    layout=widgets.Layout(width="50%")
)

Dropdown(description='Part Family:', index=10, layout=Layout(width='50%'), options=('Cover Handle Front', 'Fen…

Dropdown(description='Part No:', layout=Layout(width='50%'), options=('K2220500', 'K2221020', 'K4221620', 'K42…

,Metric,Min Val,Max Val,Delta (Max - Min)
0,PO Price,1.27,1.27,0.00
1,RM Rate,98.55,98.55,0.00
2,Gross Weight,0.01,0.01,0.00
3,Net RM Cost,0.51,0.51,0.00
4,Net BOP Cost,0.00,0.00,0.00
5,Net Process Cost,0.57,0.57,0.00
6,Net Conversion Cost,0.57,0.57,0.00
7,Overhead Cost,0.00,0.00,0.00
8,Profit Cost,0.12,0.12,0.00
9,Rejection Cost,0.01,0.01,0.00


In [14]:
METRICS = {
    "PO Price": "PO Price",
    "RMRatePerKg": "RM Rate",
    "GrossWeight": "Gross Weight",
    "Net RM Cost": "Net RM Cost",
    "Net BOP Cost": "Net BOP Cost",
    "Net Process Cost": "Net Process Cost",
    "Net Conversion Cost": "Net Conversion Cost",
    "Overhead Combined Cost": "Overhead Cost",
    "Profit Cost": "Profit Cost",
    "Rejection Cost": "Rejection Cost",
    "Packaging Cost": "Packaging Cost",
    "Freight Cost": "Freight Cost"
}

# force numeric once (important)
for col in METRICS:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [25]:
from IPython.display import HTML

def show_insights(part_family, part_no):
    clear_output(wait=True)
    display(part_family_dd, part_dd)

    df_part = df[
        (df[PART_FAMILY_COL] == part_family) &
        (df[PART_COL] == part_no)
    ]

    if df_part.empty:
        print("No data available")
        return

    # -------------------------------
    # HEADER
    # -------------------------------
    display(HTML(f"""
    <h3 style="margin-bottom:5px;">📌 Part Family: {part_family}</h3>
    <h4 style="margin-top:0;">🧩 Part No: {part_no}</h4>
    """))

    # -------------------------------
    # SUPPLIER INSIGHTS
    # -------------------------------
    supplier_count = df_part[SUPPLIER_COL].nunique()
    source_type = "Single Source" if supplier_count == 1 else "Multi Source"

    display(HTML(f"""
    <b>🏭 Supplier Count:</b> {supplier_count} &nbsp;&nbsp;
    <b>🔁 Source Type:</b> {source_type}
    <hr>
    """))

    # -------------------------------
    # METRIC INSIGHTS (MIN / MAX / DELTA)
    # -------------------------------
    insight_rows = []

    for col, label in METRICS.items():
        if col in df_part.columns:
            min_val = df_part[col].min()
            max_val = df_part[col].max()
            delta = max_val - min_val

            insight_rows.append({
                "Metric": label,
                "Min Val": min_val,
                "Max Val": max_val,
                "Delta (Max - Min)": delta
            })

    insights_df = pd.DataFrame(insight_rows)

    display(HTML("<h4>📊 Part-wise Cost Insights</h4>"))
    display(insights_df)